In [2]:
# Libraries 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
from PIL import Image


from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam
from keras.layers import Dropout

https://github.com/surmenok/GTSRB/blob/master/german-traffic-signs.ipynb
https://www.kaggle.com/code/bella1220/cnn-gridsearch-layers-batchsize-dropout/edit/run/123386426


cv2 is a Python package for computer vision tasks, specifically for real-time computer vision. It stands for "OpenCV", which stands for Open Source Computer Vision Library. OpenCV is a free and open-source library that provides tools and algorithms for processing images and videos. cv2 provides an interface for accessing these tools and algorithms through Python. It is commonly used for tasks such as object detection, image processing, and video analysis.

In [12]:
# Download and unpack the training set and the test set
! wget http://benchmark.ini.rub.de/Dataset/GTSRB_Final_Training_Images.zip -P data
! wget http://benchmark.ini.rub.de/Dataset/GTSRB_Final_Test_Images.zip -P data
! wget http://benchmark.ini.rub.de/Dataset/GTSRB_Final_Test_GT.zip -P data
! unzip data/GTSRB_Final_Training_Images.zip -d data
! unzip data/GTSRB_Final_Test_Images.zip -d data
! unzip data/GTSRB_Final_Test_GT.zip -d data

--2023-03-29 01:27:16--  http://benchmark.ini.rub.de/Dataset/GTSRB_Final_Training_Images.zip
Resolving benchmark.ini.rub.de (benchmark.ini.rub.de)... 134.147.122.15
Connecting to benchmark.ini.rub.de (benchmark.ini.rub.de)|134.147.122.15|:80... failed: Operation timed out.
Retrying.

--2023-03-29 01:28:32--  (try: 2)  http://benchmark.ini.rub.de/Dataset/GTSRB_Final_Training_Images.zip
Connecting to benchmark.ini.rub.de (benchmark.ini.rub.de)|134.147.122.15|:80... failed: Operation timed out.
Retrying.

--2023-03-29 01:29:49--  (try: 3)  http://benchmark.ini.rub.de/Dataset/GTSRB_Final_Training_Images.zip
Connecting to benchmark.ini.rub.de (benchmark.ini.rub.de)|134.147.122.15|:80... failed: Operation timed out.
Retrying.

--2023-03-29 01:31:07--  (try: 4)  http://benchmark.ini.rub.de/Dataset/GTSRB_Final_Training_Images.zip
Connecting to benchmark.ini.rub.de (benchmark.ini.rub.de)|134.147.122.15|:80... failed: Operation timed out.
Retrying.

--2023-03-29 01:32:26--  (try: 5)  http://benc

In [ ]:
# Move the test set to data/test
! mkdir data/test
! mv data/GTSRB/Final_Test/Images/*.ppm data/test

In [ ]:
# Download class names
! wget https://raw.githubusercontent.com/georgesung/traffic_sign_classification_german/master/signnames.csv -P data

In [5]:
from collections import namedtuple

Annotation = namedtuple('Annotation', ['filename', 'label'])
def read_annotations(filename):
    annotations = []
    
    with open(filename) as f:
        reader = csv.reader(f, delimiter=';')
        next(reader) # skip header

        # loop over all images in current annotations file
        for row in reader:
            filename = row[0] # filename is in the 0th column
            label = int(row[7]) # label is in the 7th column
            annotations.append(Annotation(filename, label))
            
    return annotations

In [6]:
def load_training_annotations(source_path):
    annotations = []
    for c in range(0,43):
        filename = os.path.join(source_path, format(c, '05d'), 'GT-' + format(c, '05d') + '.csv')
        annotations.extend(read_annotations(filename))
    return annotations

def copy_files(label, filenames, source, destination, move=False):
    func = os.rename if move else shutil.copyfile
    
    label_path = os.path.join(destination, str(label))
    if not os.path.exists(label_path):
        os.makedirs(label_path)
        
    for filename in filenames:
        destination_path = os.path.join(label_path, filename)
        if not os.path.exists(destination_path):
            func(os.path.join(source, format(label, '05d'), filename), destination_path)

def split_train_validation_sets(source_path, train_path, validation_path, all_path, validation_fraction=0.2):
    """
    Splits the GTSRB training set into training and validation sets.
    """
    
    if not os.path.exists(train_path):
        os.makedirs(train_path)
        
    if not os.path.exists(validation_path):
        os.makedirs(validation_path)
        
    if not os.path.exists(all_path):
        os.makedirs(all_path)
    
    annotations = load_training_annotations(source_path)
    filenames = defaultdict(list)
    for annotation in annotations:
        filenames[annotation.label].append(annotation.filename)

    for label, filenames in filenames.items():
        filenames = sorted(filenames)
        
        validation_size = int(len(filenames) // 30 * validation_fraction) * 30
        train_filenames = filenames[validation_size:]
        validation_filenames = filenames[:validation_size]
        
        copy_files(label, filenames, source_path, all_path, move=False)
        copy_files(label, train_filenames, source_path, train_path, move=True)
        copy_files(label, validation_filenames, source_path, validation_path, move=True)

In [11]:
import os 

path = 'Project/data/'

source_path = os.path.join(path, 'GTSRB/Final_Training/Images')
train_path = os.path.join(path, 'train')
validation_path = os.path.join(path, 'valid')
all_path = os.path.join(path, 'all')
validation_fraction = 0.2
split_train_validation_sets(source_path, train_path, validation_path, all_path, validation_fraction)

FileNotFoundError: [Errno 2] No such file or directory: 'Project/data/GTSRB/Final_Training/Images/00000/GT-00000.csv'

In [ ]:
test_annotations = read_annotations('data/GT-final_test.csv')

In [ ]:
classes = pd.read_csv('data/signnames.csv')
class_names = {}
for i, row in classes.iterrows():
    class_names[str(row[0])] = row[1]

In [ ]:
arch=resnet34
sz = 96
data = ImageClassifierData.from_paths(path, tfms=tfms_from_model(arch, sz), test_name='test')

In [ ]:
plt.imshow(load_img_id(data.val_ds, 1, path))
plt.show()

In [ ]:
y = data.trn_ds.y

In [ ]:
from collections import Counter

label_counts = Counter(y).most_common()
for l, c in label_counts:
    print(c, '\t', data.classes[l], '\t', class_names[data.classes[l]])

In [ ]:
for label in sorted([l for l, c in label_counts], key=lambda p: int(data.classes[p])):
    i = [i for i, l in enumerate(y) if l == label][0]
    print(data.classes[y[i]], class_names[data.classes[y[i]]])
    plt.imshow(load_img_id(data.trn_ds, i, path))
    plt.show()

In [ ]:
#importing training dataset
train=pd.read_csv('../input/gtsrb-german-traffic-sign/Train.csv')
X_train=train['Path']
y_train=train.ClassId
train

In [ ]:
len(np.unique(y_train))

In [ ]:
X_train.shape

In [ ]:
len(np.unique(X_train))

In [ ]:
data_dir = "../input/gtsrb-german-traffic-sign"
train_imgpath= list((data_dir + '/' + str(train.Path[i])) for i in range(len(train.Path)))

In [ ]:
for i in range(0,9):
    plt.subplot(331+i) # plt.subplot(nrows, ncols, index) -> 3 x 3 grid and index 0 ~ 8 
    seed=np.random.randint(0,39210)
    im = Image.open(train_imgpath[seed])  
    plt.imshow(im)
    
plt.show()

# Preprocessing image-
converting images into arrays of the form (28,28,3)

In [ ]:
len(train.Path)

In [ ]:
train_data=[]
train_labels=[]


path = "../input/gtsrb-german-traffic-sign/"

for i in range(len(train.Path)):
    image=cv2.imread(train_imgpath[i]) # Read the train_imgpath 0 ~ end 
    image_from_array = Image.fromarray(image, 'RGB') # form image from the array (The second argument 'RGB' specifies the mode of the image / 'L' for greyscale)
    size_image = image_from_array.resize((28,28)) # resize the image to (28,28)
    train_data.append(np.array(size_image)) # train_data includes array of the image size of (28,28)
    train_labels.append(train.ClassId[i]) # train_labels includes class of the each image 0 ~ end


X=np.array(train_data)
y=np.array(train_labels)

In [ ]:
#Spliting the images into train and validation sets

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split( X, y, test_size=0.20, random_state=7777)  

In [ ]:
X_train = X_train.astype('float32')/255 
X_val = X_val.astype('float32')/255
# normalizes the pixel values in the training and validation sets by dividing them by 255. 
# This is because the pixel values range from 0 to 255, and by dividing by 255 we can rescale them to range from 0 to 1,
# which can help with model training. 
# The astype('float32') function converts the pixel values from the original integer format to floating point format.

#Using one hote encoding for the train and validation labels
from keras.utils import to_categorical
y_train = to_categorical(y_train, 43)
y_val = to_categorical(y_val, 43)

# CNN Model - GridSearch

In [ ]:
# CNN Model
# Grid Search to determine the layers and neurons in each layer in the sequential model.

def create_model(layers):
    # 2 layers of Conv2D + Maxpool 
    cnn = tf.keras.models.Sequential()
    cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu", input_shape=[28, 28, 3]))
    cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))
    cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu"))
    cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))
    
    # Flatten from 3D tensor (bath_size , height, width, filters) -> 1 D tensor 
    cnn.add(tf.keras.layers.Flatten())
    
    # multiple dense layers 
    # (apply multiplication operation followed by a bias addition opperation; output = activation(dot(input, weights) + bias))
    # relu is used to introduce non-linearity to the model(learn complex features)
    #  It replaces negative input values with zero and leaves positive input values unchanged, 
    # effectively removing any linearity and allowing for the neural network to learn more complex and non-linear patterns in the data. 
    for i, nodes in enumerate(layers):
        cnn.add(tf.keras.layers.Dense(units=nodes, activation='relu'))
    
    # output layer 
    cnn.add(tf.keras.layers.Dense(units=43, activation='softmax'))
    
    cnn.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return cnn

In [ ]:
# Define the model to be used in GridSearchCV
model = KerasClassifier(build_fn=create_model, verbose=1)

# Define the list of different layers to be tried
layers = [[128],(256, 128),(200, 150, 120)]

# Define the parameter grid as a dictionary with the different layers
param_grid = dict(layers=layers)

# Initialize the GridSearchCV object with the model, parameter grid, and verbose output
grid = GridSearchCV(estimator=model, param_grid=param_grid, verbose=1)

# Fit the GridSearchCV object to the training data and specify the validation data
grid_results = grid.fit(X_train,y_train, validation_data=(X_val, y_val))

# Print the best results and the parameters that produced them
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))

In [ ]:
# Extract the mean test scores, standard deviations, and parameters from the GridSearchCV object
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']

# Iterate through the mean test scores, standard deviations, and parameters and print them
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

In [ ]:
# Extract the mean test scores, standard deviations, and parameters from the GridSearchCV object
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']

# Create a list of strings to use as xticks labels
xticks_labels = [str(param['layers']) for param in params]

# Set up the figure and axes
fig, ax = plt.subplots(figsize=(8, 6))

# Plot the mean test scores as a bar chart
ax.bar(range(len(means)), means, yerr=stds, align='center', alpha=0.5, ecolor='black', capsize=10)

# Set the xticks labels
ax.set_xticks(range(len(means)))
ax.set_xticklabels(xticks_labels)

# Set the y axis label
ax.set_ylabel('Mean test score')

# Set the title
ax.set_title('GridSearchCV Results')

# Show the plot
plt.show()

Grid Search to determine the batch size

In [ ]:
def create_model1():
    cnn = tf.keras.models.Sequential()
    cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu", input_shape=[28, 28, 3]))
    cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))
    cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu"))
    cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))
    cnn.add(tf.keras.layers.Flatten())
    cnn.add(tf.keras.layers.Dense(units=256, activation='relu'))
    cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
    cnn.add(tf.keras.layers.Dense(units=43, activation='softmax'))
    
    cnn.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return cnn

model = KerasClassifier(build_fn = create_model1, verbose = 1)

batch_size = [20,40]
param_grid = dict(batch_size=batch_size)

grid = GridSearchCV(estimator = model, param_grid = param_grid, verbose = 1)
grid_results = grid.fit(X_train,y_train, validation_data=(X_val, y_val))

print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
params = grid_results.cv_results_['params']
for mean,param in zip(means,params):
    print('{0} with: {1}'.format(mean,param))

Grid Search to determine the dropout rate

In [ ]:
def create_model2(dropout):
    # create model
    cnn = tf.keras.models.Sequential()
    cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu", input_shape=[28, 28, 3]))
    cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))
    cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu"))
    cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))
    cnn.add(tf.keras.layers.Flatten())
    cnn.add(tf.keras.layers.Dense(units=256, activation='relu'))
    cnn.add(Dropout(dropout))
    cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
    cnn.add(Dropout(dropout))
    cnn.add(tf.keras.layers.Dense(units=43, activation='softmax'))
    
    cnn.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return cnn

model = KerasClassifier(build_fn = create_model2, verbose = 1, batch_size=20)

dropout = [0.0, 0.1, 0.2]
param_grid = dict(dropout=dropout)

grid = GridSearchCV(estimator = model, param_grid = param_grid, verbose = 1)
grid_results = grid.fit(X_train,y_train, validation_data=(X_val, y_val))

print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
params = grid_results.cv_results_['params']
for mean,param in zip(means,params):
    print('{0} with: {1}'.format(mean,param))

Inputing the parameters in the final model (for better accuracy, you can run grid search multiple times zooming into each range used before)

In [ ]:
#Definition of the DNN model

cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu", input_shape=[28, 28, 3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu"))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(units=256, activation='relu'))
cnn.add(Dropout(0.1))
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(Dropout(0.1))
cnn.add(tf.keras.layers.Dense(units=43, activation='softmax'))

# compile the model
cnn.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

history = cnn.fit(X_train, y_train, batch_size=20, epochs=20,validation_data=(X_val, y_val))

Plotting the values of accuracy and loss vs epoch to visually determine the suitable number of epochs required

In [ ]:
import matplotlib.pyplot as plt

plt.figure(0)
plt.plot(history.history['accuracy'], label='training accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()

plt.figure(1)
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()

# preparing test data

In [ ]:
test=pd.read_csv('../input/gtsrb-german-traffic-sign/Test.csv')
X_test=train['Path']
y_test=train.ClassId

In [ ]:
data_dir = "../input/gtsrb-german-traffic-sign"
test_imgpath= list((data_dir + '/' + str(test.Path[i])) for i in range(len(test.Path)))

In [ ]:
test_data=[]
test_labels=[]


path = "../input/gtsrb-german-traffic-sign/"
for i in range(len(test.Path)):
    image=cv2.imread(test_imgpath[i])
    image_from_array = Image.fromarray(image, 'RGB')
    size_image = image_from_array.resize((28,28))
    test_data.append(np.array(size_image))
    test_labels.append(test.ClassId[i])


X_test=np.array(test_data)
y_test=np.array(test_labels)

X_test = X_test.astype('float32')/255 

In [ ]:
#predictions-
pred = cnn.predict_classes(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred)